In [1]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI
from pprint import pprint

In [2]:
load_dotenv(override=True)
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OPENAI_API_KEY environment variable is not set.")

In [3]:
openai = OpenAI()

In [5]:
headers = {
  "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"   
}

class Website:
    def __init__(self, url, headers=None):
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.get_text(separator=' ', strip=True)

In [6]:
ed = Website("https://edwarddonner.com", headers=headers)
print(ed.title)
print(ed.text)

Home - Edward Donner
Home - Edward Donner Home Connect Four Outsmart An arena that pits LLMs against each other in a battle of diplomacy and deviousness About Posts Well, hi there. I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production ( very amateur) and losing myself in Hacker News , nodding my head sagely to things I only half understand. I’m the co-founder and CTO of Nebula.io . We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt, acquired in 2021 . We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve patented our matching model, and our award-winning platform has happy customers and tons of p

In [7]:
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [8]:
def user_prompt(website):
    user_prompt = f"You are looking at website titled {website.title}"
    user_prompt += f"\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or annoucements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [9]:
print(user_prompt(ed))

You are looking at website titled Home - Edward Donner
The contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or annoucements, then summarize these too.

Home - Edward Donner Home Connect Four Outsmart An arena that pits LLMs against each other in a battle of diplomacy and deviousness About Posts Well, hi there. I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production ( very amateur) and losing myself in Hacker News , nodding my head sagely to things I only half understand. I’m the co-founder and CTO of Nebula.io . We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI sta

In [10]:
message = [
    {"role": "system", "content": "You are a snarky assistant"},
    {"role": "user", "content": "What is 2 + 2?"}
]

In [11]:
response = openai.chat.completions.create(model="gpt-4o-mini", messages=message)
print(response.choices[0].message.content)

Well, that's a math classic! 2 + 2 equals 4. But if you're looking for a more creative answer, I could always say "the number of times I've rolled my eyes today."


In [12]:
def message_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt(website)}
    ]

In [13]:
message_for(ed)

[{'role': 'system',
  'content': 'You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown.'},
 {'role': 'user',
  'content': 'You are looking at website titled Home - Edward Donner\nThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or annoucements, then summarize these too.\n\nHome - Edward Donner Home Connect Four Outsmart An arena that pits LLMs against each other in a battle of diplomacy and deviousness About Posts Well, hi there. I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production ( very amateur) and losing myself in Hacker News , nodding my head sagely to things I only half understand. I’m the co-founder and CTO of Nebula.io . We’re applying AI to a field where it can ma

In [14]:
def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=message_for(website)
    )
    return response.choices[0].message.content

In [15]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [16]:
display_summary("https://edwarddonner.com")

# Summary of Edward Donner's Website

Edward Donner's website serves as a personal and professional platform where he shares his interests and expertise in coding, AI, and LLMs (Large Language Models). He introduces himself as a coder and experimenter who enjoys DJing and electronic music production, as well as staying updated with tech news.

## Professional Background
- Co-founder and CTO of **Nebula.io**, focused on using AI to help individuals discover their potential.
- Previously the founder and CEO of the AI startup **untapt**, which was acquired in 2021 and specializes in talent matching using proprietary LLMs.

## Recent Posts and Announcements
1. **Connecting my courses – become an LLM expert and leader** (May 28, 2025)
2. **2025 AI Executive Briefing** (May 18, 2025)
3. **The Complete Agentic AI Engineering Course** (April 21, 2025)
4. **LLM Workshop – Hands-on with Agents – resources** (January 23, 2025)

Overall, the site highlights Edward's interests, professional journey, and ongoing educational initiatives in the field of AI and LLMs.

In [ ]:
display_summary("https://cnn.com")

In [ ]:
display_summary("https://archlinux.org/")